#Imports

In [2]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 15.3 MB/s 
     |████████████████████████████████| 19.2 MB 1.3 MB/s 
     |████████████████████████████████| 240 kB 59.9 MB/s 
     |████████████████████████████████| 112 kB 59.6 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 
     |████████████████████████████████| 49 kB 6.3 MB/s 
     |████████████████████████████████| 50 kB 7.2 MB/s 
     |████████████████████████████████| 943 kB 46.3 MB/s 
     |████████████████████████████████| 37.5 MB 1.2 MB/s 
     |████████████████████████████████| 4.0 MB 48.7 MB/s 
     |████████████████████████████████| 1.9 MB 52.5 MB/s 
     |████████████████████████████████| 112 kB 64.1 MB/s 
  Created wheel for qiskit: filename=qiskit-0.39.0-py3-none-any.whl size=12188 sha256=d590079cddf343cdb9b1c3c46a7b830aed4ecb266df0d91da2c37bc446eef784
  Stored in directory: /root/.cache/pip/wheels/63/d0/9a/2571

In [3]:
#import qiskit
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer

#import numpy
import numpy as np

# Exercise 1: multipliers

We are going to tackle this multiplication algorithm as a repeated addition problem. The steps for the implementation of this algorithm would be:

- Represent the multplier and multiplicand as quantum states. 
- Create a bitsring 0 called the accumulator. 
- Add the multiplier to the accumulator.
- Decrement the multiplier by one.
- Repeat until the multiplier is 0.
- Measure the final output (accumulator).

For this problem, we will employ quantum fourier addition and substraction.

1. The first step is to get the multiplicand and multiplier as a bit string. Numbers should be less than 9 digits as the IBM simulator only supports up to 32 qubits. If we assume that n is the length of both the numbers we want to add:
- n qubits to store the multiplier.
- n qubits to store the multiplicand.
- 2n qubits to store the accumulator.
- We need 2n classical bits to store the final results.
Therefore we have 5n qubits that we want to fit into 32, hence the maximum we can calculate is a 6-digits number. 

In [5]:
def convert_to_bitstring(int1, int2):
  """Converts two integer to bitstring format of same length"""

  #convert integers to bitstrings
  num1 = str(format(int1, "b"))
  num2 = str(format(int2, "b"))

  len1 = len(num1)
  len2 = len(num2)

  if len1 > 6 or len2 > 6:
    raise ValueError('The length of the integers in bit format should be less than 6 digits.')

  #check if both numbers have the same length
  #otherwise fill with zeros

  if len1>len2:
    num2= '0'*(len1-len2) + num2
  else:
    num1= '0'*(len2-len1) + num1

  return num1, num2

2. Store numbers in quantum states.

In [6]:
def bitstring_to_quantum_state(qc, qr, bitstring):
  """Convert bitstring to quantum state"""
  n = len(bitstring)
  for i in range(n):
    if bitstring[i] == "1":
        qc.x(qr[n-(i+1)])

3. Add multiplicand to the accumulator using addition. We are going to use quantum Fourier addition.

In [7]:
def QFT(qc, n, q_reg1):
  """Function that performs QFT to an n quantum register 
  by applying repeated rotations and hadamard gates"""

  qc.h(q_reg1[n]) #hadamard gates at the beginning of each qubit rotations

  #apply rotations
  for i in range(n):
    rotation =  np.pi / (2**(i+1))
    qc.cu1(rotation, q_reg1[n-(i+1)], q_reg1[n])


In [8]:
def modular_addition_substraction(qc, n , q_reg1, q_reg2, sign):
  """Evolves the quantum circuit to perform quantum Fourier 
  addition and substraction by applying repeated rotations"""

  N = q_reg1.size   #size of first register
  l = q_reg2.size   #size of second register

  #apply rotations
  for i in range(l):
    if n - l +1 >= 0:
      rotation = np.pi / (2**(n-l+i+1))
    
    if n - l + 1 < 0:
      #condition for registers with different sizes
      if n - i < 0:
        break
      else: 
        rotation = np.pi / (2**i)

    #condition for substraction
    if sign == '-':
      qc.cu1(-rotation,  q_reg1[n], q_reg2[-i-1])

    #condition for addition
    if sign == '+':
      qc.cu1(rotation, q_reg1[n], q_reg2[-i-1])


In [9]:
def inverse_QFT(qc, n, q_reg1):
  """Function that performs inverse QFT to an n quantum register 
  by applying repeated rotations and hadamard gates"""

  #apply rotations
  for i in range(n):
    rotation = - np.pi / (2**(n-i))
    qc.cu1(rotation, q_reg1[i], q_reg1[n])

  qc.h(q_reg1[n]) #hadamard gates at the end of each qubit rotations

In [10]:
def addition_substraction(qc, q_reg1, q_reg2, sign):
  """Function that adds two quantum registers and stores the results
  in the first register"""

  len1 = q_reg1.size  #size of the first register
  len2= q_reg2.size   #size of the second register

  # Compute the quantum Fourier transform of first register
  for i in range(len1):
      QFT(qc, len1-i-1, q_reg1)

  # Compute the addition of first and second register 
  # and store result in the first register
  for i in range(len1):
      modular_addition_substraction(qc, len1-i-1, q_reg1, q_reg2, sign)

  # Compute the inverse Fourier transform of first register
  for i in range(len1):
      inverse_QFT(qc, i, q_reg1)

4. Add the multiplier to the accumulator. Decrement the multiplier by one.Repeat until the multiplier is 0. Measure the final output (accumulator).

In [40]:
def multiplier(int1, int2):
  """
  int1 : integer positive value that is the first parameter to the multiplier function,
  int2 : integer positive value that is the second parameter to the multiplier function.
  Return the positive integer value of the multiplication between number_1 and number_2
  """ 


  #check for negative numbers
  dummy = 1
  if abs(int1) != int1 and abs(int2) == int2:
    dummy = -1
  elif abs(int2) != int2 and abs(int1) == int1:
    dummy = -1

  #convert to bitstring
  bit1, bit2 = convert_to_bitstring(abs(num1), abs(num2))
  n = len(bit1)

  #create quantum register
  accumulator = QuantumRegister(2*n, "accumulator")  #stores the final result
  multiplier = QuantumRegister(n, "multiplier")
  multiplicand = QuantumRegister(n, "multiplicand")
  x = QuantumRegister(1, "x")   #quantum register set to substract 1 to the multiplicand
  cl = ClassicalRegister(2*n, "cl")

  #create quantum circuit
  qc = QuantumCircuit(accumulator, multiplicand, multiplier, x, cl)

  #flip qubit to get the state |1>
  qc.x(x[0])

  #create quantum states for the multiplier and multiplicand
  bitstring_to_quantum_state(qc, multiplier, bit1)
  bitstring_to_quantum_state(qc, multiplicand, bit2)

  multiplier_2 = '1'
  while(int(multiplier_2) != 0):
    #add multplicand to the accumulator
    addition_substraction(qc, accumulator, multiplicand, '+')
    #substract 1 to the multiplier
    addition_substraction(qc, multiplier, x, '-')
    #measure the multiplier, if it's 0 break the loop, else continue
    for i in range(len(multiplier)):
      qc.measure(multiplier[i], cl[i])
    result = execute(qc, backend=Aer.get_backend('qasm_simulator'), shots=2).result().get_counts(qc.name)

    multiplier_2 = list(result.keys())[0]

  qc.measure(accumulator, cl)
  result = execute(qc, backend=Aer.get_backend('qasm_simulator'),
            shots=2).result().get_counts(qc.name)
  result = list(result.keys())[0]  
  return int(str(result), 2) * dummy


# Tests

In [41]:
#MULTIPLICATION OF -4 X -4
num1 = -4
num2= -4

bit1, bit2 = convert_to_bitstring(num1, num2)

print('Number ' + str(num1) + ' as bitstring is ' + bit1)
print('Number ' + str(num2) + ' as bitstring is ' + bit2)

result = multiplier(num1, num2)
print('The result of multiplying ' + str(num1)+ ' and '+ str(num2)+ ' is '+ str(result))

Number -4 as bitstring is -100
Number -4 as bitstring is -100


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: The QuantumCircuit.cu1 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use the QuantumCircuit.cp method instead, which acts identically.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: The QuantumCircuit.cu1 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use the QuantumCircuit.cp method instead, which acts identically.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: The QuantumCircuit.cu1 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use the QuantumCircuit.cp method instead, which acts identically.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarn

The result of multiplying -4 and -4 is 16


In [43]:
#MULTIPLICATION OF 4 X -5
num1 = 4
num2= -5

bit1, bit2 = convert_to_bitstring(num1, num2)

print('Number ' + str(num1) + ' as bitstring is ' + bit1)
print('Number ' + str(num2) + ' as bitstring is ' + bit2)

result = multiplier(num1, num2)
print('The result of multiplying ' + str(num1)+ ' and '+ str(num2)+ ' is '+ str(result))

Number 4 as bitstring is 0100
Number -5 as bitstring is -101


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: The QuantumCircuit.cu1 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use the QuantumCircuit.cp method instead, which acts identically.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: The QuantumCircuit.cu1 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use the QuantumCircuit.cp method instead, which acts identically.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: The QuantumCircuit.cu1 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use the QuantumCircuit.cp method instead, which acts identically.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarn

The result of multiplying 4 and -5 is -20


In [44]:
#MULTIPLICATION OF 10 X 5
num1 = 10
num2= 5

bit1, bit2 = convert_to_bitstring(num1, num2)

print('Number ' + str(num1) + ' as bitstring is ' + bit1)
print('Number ' + str(num2) + ' as bitstring is ' + bit2)

result = multiplier(num1, num2)
print('The result of multiplying ' + str(num1)+ ' and '+ str(num2)+ ' is '+ str(result))

Number 10 as bitstring is 1010
Number 5 as bitstring is 0101


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: The QuantumCircuit.cu1 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use the QuantumCircuit.cp method instead, which acts identically.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: The QuantumCircuit.cu1 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use the QuantumCircuit.cp method instead, which acts identically.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: The QuantumCircuit.cu1 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use the QuantumCircuit.cp method instead, which acts identically.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarn

The result of multiplying 10 and 5 is 54


In [25]:
#MULTIPLICATION OF 10 X 100
num1 = 10
num2= 100

bit1, bit2 = convert_to_bitstring(num1, num2)

print('Number ' + str(num1) + ' as bitstring is ' + bit1)
print('Number ' + str(num2) + ' as bitstring is ' + bit2)

result = multiplier(num1, num2)
print('The result of multiplying ' + str(num1)+ ' and '+ str(num2)+ ' is '+ str(result))

ValueError: ignored

Very interesting exercise! would have loved to try the 4th one but there was a delay of the email with the exercises so I only got a few days to complete the first one. 